# Join Pax data from Beontra into AOS data

Working on the AOS data, I realized that we need to remove some lines that do not follow the logic of "flight_data".
All flight data should end up in "flight_data" table: all airports, direction, sources (eg. AODB, forecast, etc).
The goal is to be able to easily analyze, compare and create tools from this single table. (== same format)

In "flight_data" one line = one movement (arrival or departure)
However, in AOS data, some lines do not represent a movement of an aircraft.

The Pax values are OK as merged with the logic in this notebook, however some extra step will be required.

After dividing AOS between arrivals and departures, we should try removing the following lines in the 2 tables (aos_arrival & aos_departure):
- "flight number" starting with "STAY"
- nan "link_arrival" or "link_departure"
- "link_arrival" or "link_departure" not 19 char long


## import and csv read

In [ ]:
import pandas as pd

In [ ]:
# transform to csv and remove useless header
# aos = pd.read_excel("../data/KIX_AOS_data/Actual Data_AOS_KIX_2018.xlsx",header=1)
# beontra = pd.read_excel("../data/KIX_Beontra_data/CY2018 KIX.xlsx",header=1)
# aos.to_csv("../data/KIX_AOS_data/Actual Data_AOS_KIX_2018.csv")
# beontra.to_csv("../data/KIX_Beontra_data/CY2018 KIX.csv")

In [ ]:
aos = pd.read_csv("../data/KIX_AOS_data/Actual Data_AOS_KIX_2018.csv")
beontra = pd.read_csv("../data/KIX_Beontra_data/CY2018 KIX.csv")

## Build the key to join AOS <-> Beontra

In [ ]:
# key for AOS
aos[["[Arr]ALcode", "[Arr]FLnumber", "[Arr]OpsSuffix"]] = aos["到着便航空機識別"].str.extract(
    "([A-Za-z]+)(\d*)([A-Za-z]*)", expand=True
)

aos[["[Dep]ALcode", "[Dep]FLnumber", "[Dep]OpsSuffix"]] = aos["出発便航空機識別"].str.extract(
    "([A-Za-z]+)(\d*)([A-Za-z]*)", expand=True
)
aos["link_arrival"] = (
    aos["[Arr]ALcode"]
    + aos["[Arr]FLnumber"].replace({"": 0}).fillna(0).apply(lambda x: f"{int(x):04d}")
    + aos["到着便検索用運航日"].astype("str")
    + aos["ATA"]
).str.strip()

aos["link_departure"] = (
    aos["[Dep]ALcode"]
    + aos["[Dep]FLnumber"].replace({"": 0}).fillna(0).apply(lambda x: f"{int(x):04d}")
    + aos["出発便検索用運航日"].astype("str")
    + aos["ATD"]
).str.strip()

# key for beontra
beontra[["ALcode", "FLnumber", "OpsSuffix"]] = beontra["Flight Number"].str.extract(
    "(\S*)(?>\s)(\d*)(.*)", expand=True
)


beontra["link_arrival"] = (
    beontra["Carrier Code3L"]
    + beontra["FLnumber"].replace({"": 0}).fillna(0).apply(lambda x: f"{int(x):04d}")
    + beontra["Flight Date"].apply(lambda x: x.replace("-", ""))
    + beontra["Actual Time"].apply(lambda x: x.replace(":", "")[0:4])
)

beontra["link_departure"] = beontra["link_arrival"]

In [ ]:
# join
beontra_join_arrival = beontra[
    ["PAX_L_ADULT FC", "PAX_L_CHILD FC", "PAX_L_INFANT FC", "link_arrival"]
].set_index("link_arrival")
beontra_join_departure = beontra[
    ["PAX_L_ADULT FC", "PAX_L_CHILD FC", "PAX_L_INFANT FC", "link_departure"]
].set_index("link_departure")

aos_test = aos.join(beontra_join_arrival, on="link_arrival")
aos_test.rename(
    columns={
        "PAX_L_ADULT FC": "[Arr] PaxADULT",
        "PAX_L_CHILD FC": "[Arr] PaxCHILD",
        "PAX_L_INFANT FC": "[Arr] PaxINFANT",
    },
    inplace=True,
)

aos_test = aos_test.join(beontra_join_departure, on="link_departure")
aos_test.rename(
    columns={
        "PAX_L_ADULT FC": "[Dep] PaxADULT",
        "PAX_L_CHILD FC": "[Dep] PaxCHILD",
        "PAX_L_INFANT FC": "[Dep] PaxINFANT",
    },
    inplace=True,
)

In [ ]:
# drop duplicate link_arrival and link_departure
# drop when not a correct ID (19 char) or STAY
aos_final = aos_test.drop_duplicates(subset=["link_arrival", "link_departure"])
aos_final = aos_final[
    (aos_final["link_departure"].notna())
    & (aos_final["link_arrival"].notna())
    # cannot remove those as we did not split arrivals and departures into their own lines
    # & ~(aos_final["link_departure"].str.startswith("STAY", na=False))
    # & ~(aos_final["link_arrival"].str.startswith("STAY", na=False))
    # & (aos_final["link_departure"].str.len() == 19)
    # & (aos_final["link_arrival"].str.len() == 19)
]

## Check result

In [ ]:
aos_arr_total = aos_final["[Arr] PaxADULT"].sum()
beontra_arr_total = beontra.loc[beontra["A/D"] == "A", "PAX_L_ADULT FC"].sum()
arr_total_diff = (aos_arr_total - beontra_arr_total) / beontra_arr_total

aos_int_dep = aos_final.loc[(aos["出発便Ｄ／Ｉ区分"] == "I"), "[Dep] PaxADULT"].sum()
beontra_int_dep = beontra.loc[
    (beontra["A/D"] == "D") & (beontra["Sector"] == "I"), "PAX_L_ADULT FC"
].sum()
dep_int_diff = (aos_int_dep - beontra_int_dep) / beontra_int_dep

print(
    f"for arrivals total:\n \taos = {aos_arr_total:.0f} \n \tbeontra ="
    f" {beontra_arr_total}\n \tdifference is"
    f" {(aos_arr_total-beontra_arr_total):.0f} Pax ({arr_total_diff:.7%})\n"
)
print(
    f"for departures int'l:\n \taos = {aos_int_dep:.0f} \n \tbeontra ="
    f" {beontra_int_dep}\n \tdifference is {aos_int_dep - beontra_int_dep:.0f} Pax"
    f" ({dep_int_diff:.7%})\n"
)

In [ ]:
# beontra flight that did not match oas flights
links = aos["link_arrival"].to_list() + aos["link_departure"].to_list()
unmatched = beontra[~beontra["link_arrival"].isin(links)]

with pd.option_context(
    # 'display.max_rows',
    #   None,
    "display.max_columns",
    None,
):
    display(unmatched[unmatched["PAX_L_ADULT FC"] > 5])

In [ ]:
# check duplicate values
with pd.option_context(
    # 'display.max_rows',
    #   None,
    "display.max_columns",
    None,
):  # more options can be specified also
    display(
        aos[
            (aos["link_departure"].duplicated())
            & (aos["link_departure"].notna())
            & ~(aos["link_departure"].str.startswith("STAY", na=False))
            & (aos["link_departure"].str.len() == 19)
        ]
    )